In [1]:
imd_dir = r"/home/stormej/dev/rainscale/data/rain/rain_tif_monthly"
ndvi_dir = r"/home/stormej/dev/rainscale/data/ndvi/ndvi_monthly_resampled_0.1"
lst_dir = r"/home/stormej/dev/rainscale/data/lst/lst_monthly_resampled_0.1"
output_dir = r"/home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1"

start_year = 2000
end_year = 2022

In [2]:
import os
import numpy as np
import rasterio
import joblib
from datetime import datetime
import re

def extract_filename(filepath):
    filename = os.path.basename(filepath)
    pattern = r'(ndvi|lst)_resampled_(\d{4})-(\d{2})\.tif$'  # ndvi_resampled_2000-03.tif or lst_resampled_2000-06.tif
    
    match = re.search(pattern, filename)
    if match:
        data_type, year, month = match.groups()
        try:
            return datetime(int(year), int(month), 1)
        except ValueError:
            raise ValueError(f"Invalid date in filename: {filename}")
    return None

def predict_spatial_rainfall(
    model,
    scaler,
    ndvi_files,
    lst_files,
    output_dir,
):
    os.makedirs(output_dir, exist_ok=True)
    
    for ndvi_file, lst_file in zip(ndvi_files, lst_files):
        try:
            file_date = extract_filename(ndvi_file)
        except ValueError as e:
            print(f"Skipping file {ndvi_file} due to error: {e}")
            continue
        
        with rasterio.open(ndvi_file) as ndvi_src:
            ndvi_data = ndvi_src.read(1)
            ndvi_flat = ndvi_data.flatten()
            
            shape = ndvi_data.shape
            coords = np.array([ndvi_src.xy(row,col) for row in range(shape[0]) for col in range(shape[1])])
            latitudes, longitudes = coords[:, 1], coords[:, 0]
            
        with rasterio.open(lst_file) as lst_src:
            lst_data = lst_src.read(1)
            lst_flat = lst_data.flatten()
        
        month = file_date.month
        month_sin = np.sin(2 * np.pi * month / 12)
        month_cos = np.cos(2 * np.pi * month / 12)
        month_sin_array = np.full_like(ndvi_flat, month_sin)
        month_cos_array = np.full_like(ndvi_flat, month_cos)
        
        feature_stack = np.column_stack((
            ndvi_flat,
            lst_flat,
            latitudes,
            longitudes,
            month_sin_array,
            month_cos_array,
        ))
        
        valid_mask = (~np.isnan(feature_stack).any(axis=1))
        valid_features = feature_stack[valid_mask]
        
        valid_features_scaled = scaler.transform(valid_features)
        prediction = model.predict(valid_features_scaled)
        
        rain_raster = np.full(ndvi_flat.shape, np.nan)
        rain_raster[valid_mask] = prediction
        
        output_filename = f"rainfall_{file_date.year}-{file_date.month:02d}.tif"
        output_filepath = os.path.join(output_dir, output_filename)
        
        with rasterio.open(ndvi_file) as src:
            profile = src.profile
            profile.update(dtype=rasterio.float32, count=1, compress='lzw')
        
        # Reshape the flattened array back to 2D before writing
        rain_raster = rain_raster.reshape(shape)
        
        with rasterio.open(output_filepath, 'w', **profile) as dst:
            dst.write(rain_raster.astype(rasterio.float32), 1)
            
        print(f"Processed {output_filename} and saved to {output_filepath}")

model = joblib.load("/home/stormej/dev/rainscale/data/ml/models/rainfall_model.joblib")
scaler = joblib.load("/home/stormej/dev/rainscale/data/ml/models/scaler.joblib")

ndvi_files = [os.path.join(ndvi_dir, f) for f in os.listdir(ndvi_dir) if f.endswith('.tif')]
lst_files = [os.path.join(lst_dir, f) for f in os.listdir(lst_dir) if f.endswith('.tif')]

predict_spatial_rainfall(
    model,
    scaler,
    ndvi_files,
    lst_files,
    output_dir,
)
        

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Do

Processed rainfall_2000-02.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2000-02.tif
Processed rainfall_2000-03.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2000-03.tif
Processed rainfall_2000-04.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2000-04.tif
Processed rainfall_2000-05.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2000-05.tif
Processed rainfall_2000-06.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2000-06.tif
Processed rainfall_2000-07.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2000-07.tif
Processed rainfall_2000-08.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2000-08.tif
Processed rainfall_2000-09.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainf

[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Do

Processed rainfall_2000-10.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2000-10.tif
Processed rainfall_2000-11.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2000-11.tif
Processed rainfall_2000-12.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2000-12.tif
Processed rainfall_2001-01.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2001-01.tif
Processed rainfall_2001-02.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2001-02.tif
Processed rainfall_2001-03.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2001-03.tif
Processed rainfall_2001-04.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2001-04.tif
Processed rainfall_2001-05.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainf

[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs

...

[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend

Processed rainfall_2021-06.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2021-06.tif
Processed rainfall_2021-07.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2021-07.tif
Processed rainfall_2021-08.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2021-08.tif
Processed rainfall_2021-09.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2021-09.tif
Processed rainfall_2021-10.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2021-10.tif
Processed rainfall_2021-11.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2021-11.tif
Processed rainfall_2021-12.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2021-12.tif
Processed rainfall_2022-01.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainf

[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend

Processed rainfall_2022-02.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2022-02.tif
Processed rainfall_2022-03.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2022-03.tif
Processed rainfall_2022-04.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2022-04.tif
Processed rainfall_2022-05.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2022-05.tif
Processed rainfall_2022-06.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2022-06.tif
Processed rainfall_2022-07.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2022-07.tif
Processed rainfall_2022-08.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainfall_2022-08.tif
Processed rainfall_2022-09.tif and saved to /home/stormej/dev/rainscale/data/ml/prediction/prediction_0.1/rainf

[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 138 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 200 out of 200 | elapsed:    0.0s finished
[Parallel(n_jobs=12)]: Using backend